# Environmental Sound Classification (ESC)

## IMPORTS

In [4]:
import glob
from collections import Counter
import torch
from torch.utils.data import Dataset
from pathlib import Path
import torchaudio

## DATA

In [6]:
esc50_list = [f.split("-")[-1].replace(".wav","")
for f in
glob.glob("ESC-50/audio/*.wav")]
Counter(esc50_list)
data = "/kaggle/input/environmental-sound-classification-esc-dataset/ESC-50/audio"

In [5]:
class ESC50(Dataset):
    def __init__(self, path):
        # Get the directory listing from path
        files = Path(path).glob('*.wav')
        # Iterate through the listing and create a list of tuples (filename, label)
        self.items = [(f,int(f.name.split("-")[-1].replace(".wav",""))) for f in files]
        self.length = len(self.items)
    def __getitem__(self, index):
        filename, label = self.items[index]
        waveform, _ = torchaudio.load(filename)
        return waveform, label
    def __len__(self):
        return self.length

In [8]:
dataset = ESC50(data)
waveform, label = dataset[0]
print(waveform.shape)  # Check the waveform shape
print(label)           # Check the label

torch.Size([1, 220500])
15
